### See how informative data you can retrieve from spotify api

In [26]:
from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials
from _spotify.spotipy_auth import spotipy_auth

spotipy_auth()

sp = Spotify(auth_manager=SpotifyClientCredentials())

In [69]:
### get sample idr
def get_sample_ids():
    hit_artist = sp.search('The Beatles', type = 'artist')['artists']['items'][0]
    
    ids = {}
    ids['artist'] = hit_artist['id']

    hit_album = sp.artist_albums(ids['artist'])['items'][0]
    ids['album'] = hit_album['id']

    hit_track = sp.album_tracks(ids['album'])['items'][0]
    ids['track'] = hit_track['id']

    return ids


ids = get_sample_ids()

#### Artist Data

#### what kind of information can we get from artist id ?

In [93]:
# artist basic information
sp.artist(ids['artist'])

{'external_urls': {'spotify': 'https://open.spotify.com/artist/3WrFJ7ztbogyGnTHbHJFl2'},
 'followers': {'href': None, 'total': 23974102},
 'genres': ['beatlesque',
  'british invasion',
  'classic rock',
  'merseybeat',
  'psychedelic rock',
  'rock'],
 'href': 'https://api.spotify.com/v1/artists/3WrFJ7ztbogyGnTHbHJFl2',
 'id': '3WrFJ7ztbogyGnTHbHJFl2',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5ebe9348cc01ff5d55971b22433',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab67616100005174e9348cc01ff5d55971b22433',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f178e9348cc01ff5d55971b22433',
   'width': 160}],
 'name': 'The Beatles',
 'popularity': 82,
 'type': 'artist',
 'uri': 'spotify:artist:3WrFJ7ztbogyGnTHbHJFl2'}

In [94]:
## artist albums
[i['name'] for i in sp.artist_albums(ids['artist'])['items']]

['Revolver (Super Deluxe)',
 'Get Back (Rooftop Performance)',
 'Let It Be (Super Deluxe)',
 'Abbey Road (Super Deluxe Edition)',
 'The Beatles',
 'Live At The Hollywood Bowl',
 'Let It Be... Naked (Remastered)',
 'Yellow Submarine Songtrack',
 'On Air - Live At The BBC (Vol.2)',
 '1 (Remastered)',
 'Live At The BBC (Remastered)',
 'Let It Be (Remastered)',
 'Abbey Road (Remastered)',
 'Yellow Submarine (Remastered)',
 'The Beatles (Remastered)',
 'Magical Mystery Tour (Remastered)',
 "Sgt. Pepper's Lonely Hearts Club Band (Deluxe Edition)",
 "Sgt. Pepper's Lonely Hearts Club Band (Deluxe Edition)",
 "Sgt. Pepper's Lonely Hearts Club Band (Remastered)",
 "Sgt. Pepper's Lonely Hearts Club Band (Super Deluxe Edition)"]

In [95]:
# artist related artists
[i['name'] for i in sp.artist_related_artists(ids['artist'])['artists']]

['John Lennon',
 'George Harrison',
 'Paul McCartney',
 'Ringo Starr',
 'The Beach Boys',
 'The Kinks',
 'The Hollies',
 'Bob Dylan',
 'The Byrds',
 'The Rolling Stones',
 'Buddy Holly',
 'Simon & Garfunkel',
 'Badfinger',
 'Wings',
 'Chuck Berry',
 'Donovan',
 'Jimi Hendrix',
 'Roy Orbison',
 'Eric Clapton',
 'Elvis Presley']

In [99]:
# top tracks
[i['name'] for i in sp.artist_top_tracks(ids['artist'])['tracks']]

['Here Comes The Sun - Remastered 2009',
 'Let It Be - Remastered 2009',
 'Come Together - Remastered 2009',
 'Yesterday - Remastered 2009',
 'Twist And Shout - Remastered 2009',
 'Blackbird - Remastered 2009',
 'In My Life - Remastered 2009',
 'Hey Jude - Remastered 2015',
 'I Want To Hold Your Hand - Remastered 2015',
 'Something - Remastered 2009']

#### what kind of information can we get from album id?

In [133]:
# playlist
[i['name'] for i in sp.search('1980s', type = 'playlist')['playlists']['items']]

['1980s Greatest Hits 💜',
 '1980s Rap Retrogamer',
 'Ambient 1980s',
 '1980s Rock',
 '80s Mix',
 'All Out 80s',
 "1980's Soul & RnB",
 'Top Hits of 1980',
 "1980's Alternative",
 '1980s Dance party']

In [135]:
# playlist
res = sp.album(ids['album'])
for i in res:
    if i not in ['available_markets', 'tracks']:
        print(i, res[i])

album_type album
artists [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3WrFJ7ztbogyGnTHbHJFl2'}, 'href': 'https://api.spotify.com/v1/artists/3WrFJ7ztbogyGnTHbHJFl2', 'id': '3WrFJ7ztbogyGnTHbHJFl2', 'name': 'The Beatles', 'type': 'artist', 'uri': 'spotify:artist:3WrFJ7ztbogyGnTHbHJFl2'}]
copyrights [{'text': '© 2022 Calderstone Productions Limited (a division of Universal Music Group)', 'type': 'C'}, {'text': 'This Compilation ℗ 2022 Calderstone Productions Limited (a division of Universal Music Group)', 'type': 'P'}]
external_ids {'upc': '00602448516602'}
external_urls {'spotify': 'https://open.spotify.com/album/7C221PnWhYGv8Tc0xSbfdc'}
genres []
href https://api.spotify.com/v1/albums/7C221PnWhYGv8Tc0xSbfdc
id 7C221PnWhYGv8Tc0xSbfdc
images [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b27333b139ea459f9d93c7b8fb30', 'width': 640}, {'height': 300, 'url': 'https://i.scdn.co/image/ab67616d00001e0233b139ea459f9d93c7b8fb30', 'width': 300}, {'height': 64, 'url

In [144]:
for i in sp.album(ids['album']):
    if i not in ('available_markets', 'tracks'):
        print(i, res[i])

album_type album
artists [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3WrFJ7ztbogyGnTHbHJFl2'}, 'href': 'https://api.spotify.com/v1/artists/3WrFJ7ztbogyGnTHbHJFl2', 'id': '3WrFJ7ztbogyGnTHbHJFl2', 'name': 'The Beatles', 'type': 'artist', 'uri': 'spotify:artist:3WrFJ7ztbogyGnTHbHJFl2'}]
copyrights [{'text': '© 2022 Calderstone Productions Limited (a division of Universal Music Group)', 'type': 'C'}, {'text': 'This Compilation ℗ 2022 Calderstone Productions Limited (a division of Universal Music Group)', 'type': 'P'}]
external_ids {'upc': '00602448516602'}
external_urls {'spotify': 'https://open.spotify.com/album/7C221PnWhYGv8Tc0xSbfdc'}
genres []
href https://api.spotify.com/v1/albums/7C221PnWhYGv8Tc0xSbfdc
id 7C221PnWhYGv8Tc0xSbfdc
images [{'height': 640, 'url': 'https://i.scdn.co/image/ab67616d0000b27333b139ea459f9d93c7b8fb30', 'width': 640}, {'height': 300, 'url': 'https://i.scdn.co/image/ab67616d00001e0233b139ea459f9d93c7b8fb30', 'width': 300}, {'height': 64, 'url

### Proof of Concept

let users search for album and display informatiive information

In [148]:
sp.album(ids['album']).keys()

dict_keys(['album_type', 'artists', 'available_markets', 'copyrights', 'external_ids', 'external_urls', 'genres', 'href', 'id', 'images', 'label', 'name', 'popularity', 'release_date', 'release_date_precision', 'total_tracks', 'tracks', 'type', 'uri'])

In [153]:
for i in sp.search('80s', type = 'playlist')['playlists']['items']:
    print(i['name'])

80s Mix
80s Hits - Best of the 80s
80s Hits
80s Rock Anthems
All Out 80s
80s Rock
’80s Hard Rock
80s Country
80s DANCE PARTY
’80s Jam Session


In [154]:
for i in sp.search('90s', type = 'playlist')['playlists']['items']:
    print(i['name'])

90s Mix
90s Hits - Best of the 90s
90s Country
90s Rock Anthems
All Out 90s
Best '90s Alternative Playlist Ever
90s & 2000s Hip Hop, Rap, R&B
90s Smash Hits
90s Pop Rock
90s Rap


In [156]:
for i in sp.search('70s', type = 'playlist')['playlists']['items']:
    print(i['name'])

70s Mix
70s Hits - The Biggest Hits of the 70's
70s Road Trip
All Out 70s
70's Rock Classics
70s Rock Anthems
70s Soul Classics
Soft 70s
70's Rock (Top 100)
70s Rock Drive


In [157]:
for i in sp.search('60s', type = 'playlist')['playlists']['items']:
    print(i['name'])

60s Mix
All Out 60s
60s Country
Best of 60s and 70s
60s Rock Anthems
60s Rock Drive
Soft 60s
1960s Hits
60s Love Songs
60s Rock


In [160]:
for i in sp.search('2000s', type = 'playlist')['playlists']['items']:
    print(i['name'])

2000s Mix
2000s Throwbacks (Top 100 Hits)
2000s🔥
2000s Country
2000s R&B & Hip-Hop Playlist
All Out 2000s
I Love My 2000s Hip-Hop
2000s-2010s bangers
2000s Screamo
2000s rap bangers we all know


In [161]:
for i in sp.search('2010s', type = 'playlist')['playlists']['items']:
    print(i['name'])

2010s Mix
2010s Throwbacks
2010s Country
2010s BANGERS
The Party Hits of the 2010s
2010's Hip Hop/Rap/Trap
I Love My 2010s Hip-Hop
All Out 2010s
Top Hits Of The 2010s
I Love My '10s R&B


In [162]:
eras = [
    "60s Mix", 
    "70s Mix", 
    "80s Mix", 
    "90s Mix", 
    "2000s Mix", 
    "2010s Mix"
]

In [165]:
era_ids = []
for i in eras:
    era_ids.append(sp.search(i, type = 'playlist')['playlists']['items'][0]['id'])


In [173]:
era_playlist = {}
for i in era_ids:
    era_playlist[i] = sp.playlist_items(i)['items']

In [195]:
pprint(era_ids)

['37i9dQZF1EQoHemmkbENDL',
 '37i9dQZF1EQpVaHRDcozEz',
 '37i9dQZF1EQqZlCxLOykhS',
 '37i9dQZF1EQn2GRFTFMl2A',
 '37i9dQZF1EQn4jwNIohw50',
 '37i9dQZF1EQqedj0y9Uwvu']


In [196]:
era_ids = [
    '37i9dQZF1EQoHemmkbENDL',
    '37i9dQZF1EQpVaHRDcozEz',
    '37i9dQZF1EQqZlCxLOykhS',
    '37i9dQZF1EQn2GRFTFMl2A',
    '37i9dQZF1EQn4jwNIohw50',
    '37i9dQZF1EQqedj0y9Uwvu'
]